## Job Web-Scraping
Note: This is intented for practice to extract real time job descriptions. Please adhere to the Robots.txt

In [1]:
#If you are using Selenium for the first time, please download the webdriver and note the filepath
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time
from tqdm import tqdm

#used as a import for dateing the CSV
import datetime

#NLP libaries
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pyLDAvis.gensim
from gensim.models import LdaModel
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\scipy\sparse\sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
path_to_chromedriver = "./chromedriver_win32/chromedriver.exe"

In [3]:
#setup browser window
browser = webdriver.Chrome(executable_path= path_to_chromedriver)

In [4]:
#adds the data in URL format by removing any white space and adding the data. 
def Keyword_to_url(kw):
    kw = kw.strip(" ").replace(" ","%20")
    return kw

In [5]:
# this will be used as the URL input for multiple job searches. 
occupation = str(input("Job you are looking for: "))
location = str(input("Where do you want to look: "))
radius = str("5")
page_sort = "sortcolumns=accquisitiondate&sortdirections=DSC"
page_size = "100" #how many pages would you like to get. 
source = 'IN' #stands for indeed

Job you are looking for Data Scientist
Where do you want to look Austin, TX


In [6]:
#give it the site
url = 'https://www.careeronestop.org/Toolkit/Jobs/find-jobs.aspx?keyword='+ \
    Keyword_to_url(occupation)+\
    '&ajax=0&location='+location+'&radius='+radius+ \
    '&source='+ source +'&pagesize='+page_size+'&sortcolumns=accquisitiondate&sortdirections=DSC'
browser.get(url)
time.sleep(1)

In [43]:
#these catch the title of the job details. 
company = []
position = []
location = []
date_posted = []
pos_description = []

In [44]:
listing_num = len(browser.find_elements_by_xpath('//*[@data-title="Job Title"]'))//25

In [45]:
job_links = browser.find_elements_by_xpath('//*[@data-title="Job Title"]')

In [46]:
success = 0
check =  []

In [47]:
range(listing_num-1)

range(0, 3)

In [48]:
for job in tqdm(range(listing_num-1)):
    #Retrieve the informaiton from the job table. 
    position.append(job_links[job].text)
    company.append(browser.find_elements_by_xpath('//*[@data-title="Company"]')[job].text)
    location.append(browser.find_elements_by_xpath('//*[@data-title="Location"]')[job].text)
    date_posted.append(browser.find_elements_by_xpath('//*[@data-title="Date Posted"]')[job].text)

    #Changes the window to the new tab.
    try:
        job_links[job].click()
        browser.switch_to_window(browser.window_handles[-1])
        while len(browser.window_handles) < 2:
            time.sleep(np.random.randint(2,7))
            job_links[job].click()
        browser.switch_to_window(browser.window_handles[1])
        pos_description.append((browser.find_element_by_xpath('//*[@class="snip"]').text).replace("\n", " "))

        #close the window and open original window
        browser.close()
        browser.switch_to_window(browser.window_handles[0])
        success += 1
    except:
        check.append(job)
        pass
    

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.47s/it]


In [49]:
len(company), len(position) , len(location), len(date_posted), len(pos_description)

(3, 3, 3, 3, 3)

In [50]:
pos_description

["Job Responsibilities Develop and maintain client relationships Manage energy, environmental siting/permitting projects Lead multiple project teams across simultaneous assignments Manage the quality, timeliness and financial aspects of projects Supervise and mentor scientists, planners, engineers, designers and technicians Lead, manage and conduct studies and investigations for the purpose of preparing reports and applications for environmental permits, in support of construction projects for various industries and markets Develop protocol and processes for leading a team to collect, analyze, manage, and report resource level data for technical reports and permit applications Communicate scientific and technical information to the project team, client organizations and resource agencies through written reports and/or presentations using a careful blend of business and technical writing Provide technical and regulatory guidance to the project team, client organizations and agencies in 

In [51]:
data = pd.DataFrame({'company': company , 
                     'position': position, 
                     'location':location,
                     'date_posted':date_posted,
                     'position_description':pos_description})

In [52]:
data.head()

,company,date_posted,location,position,position_description
0,BGE Inc.,06/19/2018,"Austin, TX",Senior Environmental Project Manager,Job Responsibilities Develop and maintain clie...
1,BGE Inc.,06/19/2018,"Austin, TX",Senior Environmental Scientist,Job Responsibilities Manages and conducts stud...
2,BGE Inc.,06/19/2018,"Austin, TX",Environmental Scientist,Job Description: Conducts studies and investig...


In [53]:
browser.close()

In [54]:
data.to_csv("./data/"+str(datetime.datetime.today())[:10]+"job_list.csv")

## Using SpaCy for NLP - this is a Work in Progress
Objective: Setup for topic modeling and use LDA to determine feature importance

In [55]:
parser = spacy.lang.en.English()
nlp = spacy.load('en')

In [57]:
lib = []
for i in pos_description:
    lib.append(i)

In [58]:
parsed = []
for sentence in lib:
    parsed.append(nlp(sentence))

In [59]:
lemma = []

In [60]:
for docs in parsed:
    for token in docs:
        lemma.append(token.lemma_)

In [62]:
lemma

['job',
 'responsibilities',
 'develop',
 'and',
 'maintain',
 'client',
 'relationship',
 'manage',
 'energy',
 ',',
 'environmental',
 'siting',
 '/',
 'permit',
 'project',
 'lead',
 'multiple',
 'project',
 'team',
 'across',
 'simultaneous',
 'assignment',
 'manage',
 'the',
 'quality',
 ',',
 'timeliness',
 'and',
 'financial',
 'aspect',
 'of',
 'project',
 'supervise',
 'and',
 'mentor',
 'scientist',
 ',',
 'planner',
 ',',
 'engineer',
 ',',
 'designer',
 'and',
 'technician',
 'lead',
 ',',
 'manage',
 'and',
 'conduct',
 'study',
 'and',
 'investigation',
 'for',
 'the',
 'purpose',
 'of',
 'prepare',
 'report',
 'and',
 'application',
 'for',
 'environmental',
 'permit',
 ',',
 'in',
 'support',
 'of',
 'construction',
 'project',
 'for',
 'various',
 'industry',
 'and',
 'market',
 'develop',
 'protocol',
 'and',
 'process',
 'for',
 'lead',
 'a',
 'team',
 'to',
 'collect',
 ',',
 'analyze',
 ',',
 'manage',
 ',',
 'and',
 'report',
 'resource',
 'level',
 'datum',
 'for